Installing TTS library

In [ ]:
!pip install TTS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.7/736.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.1/581.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━

In [2]:
from TTS.api import TTS

Loading Dialogue script generated from our step 1

In [3]:
prompt_path = "prompt1.txt"
with open(prompt_path, "r") as f:
    prompt = f.read()

Separating characters dialogues

In [22]:
actors = ["tyrion lannister", "eddard stark"]

dialogues = prompt.split("\n")

for line_no,dialog in enumerate(dialogues):
    colon_index = dialog.find(":")
    if len(dialog)==0:
        dialogues.pop(line_no)

scripts = {i:{} for i in actors}
for line_no,dialog in enumerate(dialogues):
    # print(dialog)
    colon_index = dialog.find(":")
    actor_name = dialog[:colon_index].lower().strip()
    
    # print(actor_name)
    if actor_name in actors:
        scripts[actor_name][line_no] = dialog[colon_index+1:]

tl_prompt = scripts["tyrion lannister"]
es_prompt = scripts["eddard stark"]

In [23]:
tl_prompt

{1: " He wouldn't be a Targaryen without a Targaryen.",
 3: ' So, his campaign against the Starks would probably win him no more than an entire city.',
 5: ' Of course.',
 6: ' What is it about Ramsay?',
 8: ' We did anything to stop him?',
 10: " Why should I care what he wants? I'm Tigard Stark. I can help you.",
 12: ' Stop it.',
 14: " I can't.",
 16: " It's not an alliance. It's an effort to defeat Ser Loras Tyrell. A war.",
 18: ' I did not.'}

In [24]:
es_prompt

{0: " We've always known the king would oppose any attempt to undermine our military alliance.",
 2: ' You would be in a position to convince him.',
 4: ' Perhaps.',
 7: " He didn't want us to do anything to stop him.",
 9: ' He wanted me dead.',
 11: " I can. Please, don't do it.",
 13: ' Our alliances are best defended by our friends.',
 15: ' Tell me what you want.',
 17: ' Do you know who ordered that attack?'}

In [25]:
# Running a multi-speaker and multi-lingual model

# List available 🐸TTS models and choose the first one
model_name = TTS.list_models()[0]
# Init TTS
tts = TTS(model_name)

No API token found for 🐸Coqui Studio voices - https://coqui.ai 
Visit 🔗https://app.coqui.ai/account to get one.
Set it as an environment variable `export COQUI_STUDIO_TOKEN=<token>`

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_am

Getting audio for each dialogue which is step 2
Here we proposed that we will be using DC-TTS model which will be trained by us, but as we explained about the complications faced during training in report we used our alternative to complete the pipeline.

In [41]:
import os

audio_path = "audios/"
if not os.path.exists(audio_path): os.mkdir(audio_path)
actor_speaker_map = {"tyrion lannister":[tl_prompt,tts.speakers[3],'tl'],"eddard stark":[es_prompt,tts.speakers[4],'es']}
for idx,(actor) in enumerate(actors):
  for key,value in actor_speaker_map[actor][0].items():
    tts.tts_to_file(text=value, speaker=actor_speaker_map[actor][1], language=tts.languages[0], file_path=audio_path + "{}_".format(key)+actor_speaker_map[actor][2]+".wav")

 > Text splitted to sentences.
["He wouldn't be a Targaryen without a Targaryen."]
 > Processing time: 0.6104419231414795
 > Real-time factor: 0.23688083940298002
 > Text splitted to sentences.
['So, his campaign against the Starks would probably win him no more than an entire city.']
 > Processing time: 1.1690030097961426
 > Real-time factor: 0.23189902991393424
 > Text splitted to sentences.
['Of course.']
 > Processing time: 0.23446297645568848
 > Real-time factor: 0.17432191558043753
 > Text splitted to sentences.
['What is it about Ramsay?']
 > Processing time: 0.39939117431640625
 > Real-time factor: 0.20619059076737545
 > Text splitted to sentences.
['We did anything to stop him?']
 > Processing time: 0.37270331382751465
 > Real-time factor: 0.19898735388548566
 > Text splitted to sentences.
['Why should I care what he wants?', "I'm Tigard Stark.", 'I can help you.']
 > Processing time: 1.0525469779968262
 > Real-time factor: 0.19337625904773587
 > Text splitted to sentences.
['

### Wav2Lip step 3

Load Codebase

In [30]:
!git clone https://github.com/anand-vishwakarma-1/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 341, done.
remote: Total 341 (delta 0), reused 0 (delta 0), pack-reused 341
Receiving objects: 100% (341/341), 512.25 KiB | 20.49 MiB/s, done.
Resolving deltas: 100% (186/186), done.


Install dependencies

In [31]:
!cd Wav2Lip && pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 81.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.0.25 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72)
ERROR: No matching distribution found for opencv-python==4.1.0.25


Copy face detection model

In [32]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

--2023-05-17 05:33:52--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’

Wav2Lip/face_detect 100%[===================>]  85.68M  18.3MB/s    in 4.9s    

2023-05-17 05:33:58 (17.7 MB/s) - ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]



Copy wav2lip trained model

In [34]:
!cp -ri "/content/drive/MyDrive/DL/Dataset and models/wav2lip_gan.pth" /content/Wav2Lip/checkpoints/

Copying character potraits

In [36]:
!cp -ri "/content/drive/MyDrive/DL/Dataset and models/es.jpg" /content
!cp -ri "/content/drive/MyDrive/DL/Dataset and models/tl.jpg" /content
!cp -ri "/content/drive/MyDrive/DL/Dataset and models/js.jpg" /content

Creating video for each audio dialogue generated in step 2

In [45]:
img_actor_map = {"tl" :"tl.jpg","es":"es.jpg", "js": "js.jpg"}
for files in os.listdir(audio_path):
  if files.endswith(".wav"):
    indx = files.find("_")
    print(indx,files,files[indx+1:-4],files[:indx])
    img_file = img_actor_map[files[indx+1:-4]]
    print(img_file)

    cmd_str = "cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face ../" + img_file + f" --audio ../{audio_path}" + files + " --outfile results/" + files[:indx] +".mp4"
    try:
      !{cmd_str}
    except:
      print("failed")

2 17_es.wav es 17
es.jpg
Using cpu for inference.
Reading video frames...
Number of frames available for inference: 1
(80, 157)
Length of mel chunks: 46
  0% 0/1 [00:00<?, ?it/s]
  0% 0/1 [00:00<?, ?it/s]
100% 1/1 [00:20<00:00, 20.76s/it]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100% 1/1 [00:29<00:00, 29.48s/it]
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --ena

In [47]:

!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Compiling final result file

In [49]:
import re
import glob
import ffmpeg

video_files_path = "/content/Wav2Lip/results"

video_file_list = glob.glob(f"{video_files_path}/*.mp4")

loaded_video_list = []


def sorted_nicely(l):
    """ Sort the given iterable in the way that humans expect."""
    def convert(text): return int(text) if text.isdigit() else text

    def alphanum_key(key): return [convert(c)
                                   for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)


video_file_list = sorted_nicely(video_file_list)

merged_video_name = "sample"

merged_video_path = os.path.join(
    video_files_path, f"{merged_video_name}.mp4")

merged_video_list_name = os.path.join(
    video_files_path, f"{merged_video_name}_video_list.txt")

with open(merged_video_list_name, 'w') as f:
    for video in video_file_list:
        print(f"file {video}", file=f)
        print(f"file {video}")

f.close()

ffmpeg.input(merged_video_list_name,
             format='concat', safe=0).output(merged_video_path, c='copy').run()

file /content/Wav2Lip/results/0.mp4
file /content/Wav2Lip/results/1.mp4
file /content/Wav2Lip/results/2.mp4
file /content/Wav2Lip/results/3.mp4
file /content/Wav2Lip/results/4.mp4
file /content/Wav2Lip/results/5.mp4
file /content/Wav2Lip/results/6.mp4
file /content/Wav2Lip/results/7.mp4
file /content/Wav2Lip/results/8.mp4
file /content/Wav2Lip/results/9.mp4
file /content/Wav2Lip/results/10.mp4
file /content/Wav2Lip/results/11.mp4
file /content/Wav2Lip/results/12.mp4
file /content/Wav2Lip/results/13.mp4
file /content/Wav2Lip/results/14.mp4
file /content/Wav2Lip/results/15.mp4
file /content/Wav2Lip/results/16.mp4
file /content/Wav2Lip/results/17.mp4
file /content/Wav2Lip/results/18.mp4


(None, None)

In [59]:
!zip -r final_results.zip /content/Wav2Lip/results 

  adding: content/Wav2Lip/results/ (stored 0%)
  adding: content/Wav2Lip/results/17.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/5.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/0.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/8.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/2.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/7.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/12.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/3.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/14.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/10.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/.ipynb_checkpoints/ (stored 0%)
  adding: content/Wav2Lip/results/18.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/9.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/6.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/sample_video_list.txt (deflated 86%)
  adding: content/Wav2Lip/results/11.mp4 (deflated 1%)
  adding: content/Wav2Lip/results/sample.mp4 (deflated

In [60]:
!cp -ri final_results.zip "/content/drive/MyDrive/DL/Dataset and models/"

In [61]:
!zip -r tts_step2_output.zip /content/audios

updating: content/audios/ (stored 0%)
updating: content/audios/17_es.wav (deflated 39%)
updating: content/audios/3_tl.wav (deflated 21%)
updating: content/audios/13_es.wav (deflated 27%)
updating: content/audios/2_es.wav (deflated 35%)
updating: content/audios/0_es.wav (deflated 22%)
updating: content/audios/12_tl.wav (deflated 57%)
updating: content/audios/15_es.wav (deflated 48%)
updating: content/audios/6_tl.wav (deflated 38%)
updating: content/audios/11_es.wav (deflated 53%)
updating: content/audios/16_tl.wav (deflated 42%)
updating: content/audios/14_tl.wav (deflated 55%)
updating: content/audios/4_es.wav (deflated 54%)
updating: content/audios/10_tl.wav (deflated 43%)
updating: content/audios/7_es.wav (deflated 32%)
updating: content/audios/1_tl.wav (deflated 33%)
updating: content/audios/9_es.wav (deflated 43%)
updating: content/audios/18_tl.wav (deflated 48%)
updating: content/audios/5_tl.wav (deflated 52%)
updating: content/audios/8_tl.wav (deflated 42%)


In [58]:
!cp -ri tts_step2_output.zip "/content/drive/MyDrive/DL/Dataset and models/"